## 에이전트 노트북 실행

### 시나리오 컨텍스트


# !Cloud9에서 agent 코드 실행

#### Deprecated
~### 태스크 1.1: Boto3 설치 및 필수 Python 라이브러리 가져오기~

In [ ]:
%pip install -r requirements.txt

In [2]:
from botocore.exceptions import ClientError

### 태스크 1.2: Boto3 객체 및 도우미 함수 구성 

In [3]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

에이전트 이름으로 에이전트 ID를 찾는 데 사용할 도우미 함수를 설정합니다.

In [4]:
def find_agent_id_by_agent_name(client, agent_name):
    try:
        response = client.list_agents()
        agents = response.get('agentSummaries', [])
    except ClientError as e:
        print(f"Error listing agents: {e}")
        return []
    agent_id = ''
    for agent in agents:
        if agent['agentName'] == agent_name:
            agent_id = agent['agentId']
    return agent_id

In [5]:
# Invoke the helper function to find agent IDs
agent_id = find_agent_id_by_agent_name(bedrock_agent_client, 'shopping-assistant')
agent_id_with_guardrail = find_agent_id_by_agent_name(bedrock_agent_client, 'shopping-assistant-with-guardrails')

# Get the agent alias ID
try:
    # Try to get the alias ID for the shopping-assistant agent
    response = bedrock_agent_client.list_agent_aliases(agentId=agent_id)
    if response.get('agentAliasSummaries'):
        alias_id = response['agentAliasSummaries'][0]['agentAliasId']
    else:
        # If no alias found for shopping-assistant, try with the guardrail agent
        response = bedrock_agent_client.list_agent_aliases(agentId=agent_id_with_guardrail)
        if response.get('agentAliasSummaries'):
            alias_id = response['agentAliasSummaries'][0]['agentAliasId']
        else:
            # Default fallback if no aliases found
            alias_id = "DRAFT"
except Exception as e:
    print(f"Error getting agent alias: {e}")
    # Use DRAFT as fallback - this is a common default alias name in Bedrock
    alias_id = "DRAFT"

print(f"Using agent alias ID: {alias_id}")

# the below s3 bucket will be used internally to store logs for bedrock
bedrock_logging_bucket="bedrock-logging-"+region+"-"+account_id
recent_bedrock_log_file_copy='recent_bedrock_log_file_copy'

Using agent alias ID: TSTALIASID


Bedrock 에이전트를 간접적으로 호출하고 메모리를 캡처하는 데 사용할 도우미 함수를 생성합니다.

In [6]:
def invokeAgent(query, session_id, memory_id, agent_id, enable_trace=False, session_state=dict(), end_session=False):
    
    if memory_id == "":
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession= end_session,
            sessionState=session_state
        )
    else:
    # invoke the agent API
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id, 
            memoryId=memory_id,
            sessionId=session_id,
            enableTrace=enable_trace, 
            endSession= end_session,
            sessionState=session_state
        )
    
    if enable_trace:
        # Create a safe copy of the response for logging, handling datetime objects
        try:
            # Log only the metadata, not the event stream
            safe_response = {k: str(v) for k, v in agentResponse.items() if k != 'completion'}
            logger.info(f"Agent response metadata: {safe_response}")
        except Exception as e:
            logger.info(f"Could not print full response due to: {str(e)}")
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    # Handle potential datetime objects in trace
                    try:
                        trace_str = json.dumps(event['trace'], default=lambda o: str(o), indent=2)
                        logger.info(trace_str)
                    except Exception as e:
                        logger.info(f"Could not print trace due to: {str(e)}")
                        logger.info(str(event['trace']))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        if str(e).find("throttlingException"): 
            print("\033[1mThrottlingException:\033[;7m\nA throttling error occurred when calling the InvokeAgent operation. Please wait up to 60 seconds and retry this cell.\n\033[0m")

### 태스크 1.3: 가드레일을 사용하지 않고 Bedrock 에이전트와 상호 작용  

Bedrock 에이전트와 상호 작용 및 채팅을 할 때는 다음의 2가지 구성 요소를 사용하여 상태를 유지할 수 있습니다.

**session_id** : 여러 가지 질문을 하는 에이전트와의 대화를 나타냅니다. 특정 에이전트와 같은 대화를 계속 진행하려는 경우 요청에서 같은 session_id 값을 사용.  

**memory_id** : 각 사용자의 대화 기록과 컨텍스트가 안전하게 저장되는 위치.  

In [7]:
session_id:str = str(uuid.uuid4())
print("Session id="+session_id)
memory_id_1:str = str(uuid.uuid4())
print("Memory id="+memory_id_1)

Session id=2bd3599d-9be7-47bb-a39d-f7e4963bdce5
Memory id=be5ef40a-b69f-4e03-98dd-b6843c8896b1


In [8]:
q="안녕하세요, 귀사의 제품에 관심이 있습니다. 귀사가 판매하는 제품은 누가 제조하나요?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

Question: 안녕하세요, 귀사의 제품에 관심이 있습니다. 귀사가 판매하는 제품은 누가 제조하나요?

Response: 안녕하세요! 저희 회사의 제품은 주로 두 개의 제조사에서 생산됩니다:

1. AnyCompany Outdoor Power Equipment: 이 제조사는 다음 제품들을 생산합니다:
- 스트링 트리머
- 헤지 트리머
- 리프 블로워

2. AnyCompany LawnCare Solutions: 이 제조사는 다음 제품들을 생산합니다:
- 라이딩 잔디깎기 기계
- 에어레이터

모든 제품은 배터리 작동 또는 가스 엔진 방식으로, 정원 관리에 도움을 주는 다양한 장비들입니다.



계속해서 제품 관련 질문을 합니다. 각 질문을 실행한 다음 응답을 검토합니다. 

In [9]:
q="비용은 얼마입니까?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

Question: 비용은 얼마입니까?

Response: 다음은 각 제품의 가격입니다:

AnyCompany Outdoor Power Equipment 제품:
- 스트링 트리머 (P001): $50
- 헤지 트리머 (P002): $60
- 리프 블로워 (P003): $70

AnyCompany LawnCare Solutions 제품:
- 라이딩 잔디깎기 기계 (Y001): $1,200
- 에어레이터 (Y002): $800

제품마다 가격대가 다양하니, 원하시는 제품의 예산과 용도를 고려해 선택하시기 바랍니다.



수학 계산을 사용하라는 메시지가 에이전트에게 전달되도록, 다른 질문을 합니다. 

In [10]:
q="스트링 트리머 2개 구매시 비용은 얼마인가요?"
print("\033[1mQuestion:\033[0m "+q+"\n")
try: 
    response = invokeAgent(q, session_id,memory_id_1, agent_id)
    print("\033[1mResponse:\033[0m "+response+"\n")
except Exception as e:
    print(f"Error: {str(e)}")

Question: 스트링 트리머 2개 구매시 비용은 얼마인가요?

Response: 스트링 트리머 2개의 총 비용은 $100입니다. 각 스트링 트리머의 가격이 $50이므로, 2개를 구매하면 $50 × 2 = $100가 됩니다.

